# Upload ISBN CSV → Query OCLC KB API → kb:collection_name

This notebook:
1. Lets you upload a CSV of ISBNs.
2. Queries the OCLC Knowledge Base API for each ISBN.
3. Extracts **kb:collection_name** only (pipe-separated if multiple).
4. Writes results to a CSV.

## Prerequisites
- Your **OCLC WSKey** authorized for the KB API.
- A CSV with a column of ISBNs (default column: `isbn`).


Setup & Imports

In [ ]:

# If needed, install dependencies (Colab generally has these preinstalled)
# !pip install requests pandas

import os
import time
import requests
import pandas as pd
from google.colab import files


Configuration

In [ ]:

# Your OCLC WSKey
WSKEY = "[Enter WSKEY for OCLC Knowledge Base API]"

# Input CSV file containing ISBNs
#INPUT_CSV = "/content/drive/MyDrive/isbn.csv" - only use if have file in drive
ISBN_COLUMN = "isbn"

# Output CSV file
OUTPUT_CSV = "/content/drive/MyDrive/kb_collections_output.csv"

# KB API base URL
KB_API_URL = "http://worldcat.org/webservices/kb/rest/entries/search"

# Request settings
REQUEST_TIMEOUT = 30
SLEEP_BETWEEN_CALLS = 0.3  # seconds (polite pause)


Colab upload cell - pick your ISBN csv

In [ ]:

print("📤 Please select your ISBN CSV file...")
uploaded = files.upload()  # Opens the Colab file picker

# Read the first uploaded CSV into a DataFrame
uploaded_df = None
for filename in uploaded.keys():
    print(f"✅ Uploaded: {filename} ({len(uploaded[filename])} bytes)")
    # Read CSV; enforce string dtype for ISBN, strip white space
    try:
        df = pd.read_csv(filename, dtype=str)
        if ISBN_COLUMN not in df.columns:
            raise ValueError(
                f"Column '{ISBN_COLUMN}' not found. Columns: {df.columns.tolist()}"
            )
        df[ISBN_COLUMN] = df[ISBN_COLUMN].astype(str).str.strip()
        df = df.dropna(subset=[ISBN_COLUMN])
        uploaded_df = df
        print(f"✅ Loaded {len(df)} rows. Ready to query.")
    except Exception as e:
        print(f"❌ Could not read CSV: {e}")
    # Process only the first file
    break

if uploaded_df is None:
    raise RuntimeError("No valid CSV uploaded or missing required column.")

Function to query KB API (returns only kb:collection_name)

In [ ]:

def fetch_kb_collection_names(isbn: str, wskey: str) -> list:
    """
    Given an ISBN, call the OCLC KB API and return a list of kb:collection_name values.
    Returns [] if none found or on error.
    """
    params = {
        "isbn": isbn,
        "alt": "json",
        "wskey": wskey,
    }
    try:
        resp = requests.get(KB_API_URL, params=params, timeout=REQUEST_TIMEOUT)
        if resp.status_code != 200:
            # Not fatal; just report and return empty
            print(f"[WARN] ISBN {isbn} → HTTP {resp.status_code}")
            return []
        data = resp.json()
        entries = data.get("entries", [])
        names = []
        seen = set()
        for entry in entries:
            cname = entry.get("kb:collection_name")
            if cname and cname not in seen:
                names.append(cname)
                seen.add(cname)
        return names
    except requests.RequestException as e:
        print(f"[ERROR] ISBN {isbn} → {e}")
        return []


In [ ]:

def fetch_kb_collection_names(isbn: str, wskey: str) -> list:
    """
    Given an ISBN, call the OCLC KB API and return a list of kb:collection_name values.
    Returns [] if none found or on error.
    """
    params = {
        "isbn": isbn,
        "alt": "json",
        "wskey": wskey,
    }
    try:
        resp = requests.get(KB_API_URL, params=params, timeout=REQUEST_TIMEOUT)
        if resp.status_code != 200:
            # Not fatal; just report and return empty
            print(f"[WARN] ISBN {isbn} → HTTP {resp.status_code}")
            return []
        data = resp.json()
        entries = data.get("entries", [])
        names = []
        seen = set()
        for entry in entries:
            # Keep only kb:collection_name
            cname = entry.get("kb:collection_name")
            if cname and cname not in seen:
                names.append(cname)
                seen.add(cname)
        return names
    except requests.RequestException as e:
        print(f"[ERROR] ISBN {isbn} → {e}")
        return []


Query all uploaded ISBNs and save output

In [ ]:

results = []

isbns = uploaded_df[ISBN_COLUMN].tolist()
for i, isbn in enumerate(isbns, start=1):
    if not isbn:
        continue
    names = fetch_kb_collection_names(isbn, WSKEY)
    results.append({
        "isbn": isbn,
        "kb:collection_name": "|".join(names) if names else None  # pipe-separated if multiple
    })
    time.sleep(SLEEP_BETWEEN_CALLS)
    if i % 25 == 0:
        print(f"Processed {i} ISBNs...")

output_df = pd.DataFrame(results)
output_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Wrote {len(output_df)} rows to {OUTPUT_CSV}")

# Show a preview and offer file for download
display(output_df.head())
files.download(OUTPUT_CSV)
